In [35]:
import pandas as pd
import hvplot.pandas
import panel as pn
import geoviews as gv
import geopandas as gpd
import cartopy.crs as ccrs

pn.extension()




# Geospatial Data with Geoviews, Holoviz Hvplot and Panel

This is an example using the $CO_2$ emissions data from https://ourworldindata.org/ along with geopandas geometry. First I will show you how to combine the two dataframes to create a geoview graph of the country data, adding an interactive slider and drop box to filter the year and emission source. Next will be to create a graph showing the data combined for each continent.

In [2]:
''' read co2 dataset '''
df = pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv')
# Fill NAs with 0s
df.fillna(0, inplace=True)

## geopandas datasets
There are three datasetsavailable to create a map:

* **'nybb'**: Borough boundaries of New York City, data provided Department of City Planning (DCP), https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm  
* **naturalearth_cities**: capital cities, based on http://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-populated-places/  
* **naturalearth_lowres**: country boundaries, based on http://www.naturalearthdata.com/downloads/110m-cultural-vectors/110m-admin-0-countries/


Other data can be found from https://www.naturalearthdata.com/downloads/

for this example we will be using the naturalearth_lowres dataset

In [3]:
geopandas.datasets.available

['naturalearth_lowres', 'naturalearth_cities', 'nybb']

In [4]:
''' Get the data from naturalearth_lowres datset '''
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
world.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,53950935,Africa,Tanzania,TZA,150600.0,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253,Africa,W. Sahara,ESH,906.5,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


### CO2 dataset

Under the 'country' column there are not only the country names but continents, world and other sample types. First we will designate the samples into different lists so we can filter the data as needed. 

In [5]:
''' get the coutries from the co2 dataset '''
# Columns to use for display
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antartica']
income = ['Lower-middle-income countries', 'Low-income countries', 'Upper-middle-income countries', 'High-income countries']
countries = list(set(df.country.unique().tolist())-set(continents)-set(income)-set(['North America (excl. USA)','EU-27', 'Kuwaiti Oil Fires',
                                                                                    'International transport', 'European Union (28)', 'European Union (27)',
                                                                                    'Europe (excl. EU-27)','Europe (excl. EU-28)','Asia (excl. China & India)']))
display_co2 = ['country','co2','coal_co2',
       'flaring_co2', 'gas_co2', 'oil_co2',
       'other_industry_co2',]
display_co2_per_capita = ['country','co2_per_capita','coal_co2_per_capita',
       'flaring_co2_per_capita', 'gas_co2_per_capita', 'oil_co2_per_capita',
       'other_co2_per_capita']

df_country = df[df['country'].isin(countries)]

In [6]:
''' rename the world column that includes the countries '''
world.rename(columns = {'name':'country'}, inplace = True)

### Check countries between datasets

Before combining the datasets we need to check that each country is represented in the $CO_2$ dataset correctly. This means manually checking the differences between each set of country names. 

As this is an example I will only rename the columns that are incorrect, any excluded countries will not be added but if this was for a project it would be necessary to ensure each country is represented.

In [13]:
''' check the differences between each dataset '''
set_countries = set(df_country.country.unique())
set_world = set(world.country.unique())

print("df_country unique: ",sorted(set_countries.difference(set_world)),"\n")
print("world unique: ",sorted(set_world.difference(set_countries)))

df_country unique:  ['Andorra', 'Anguilla', 'Antigua and Barbuda', 'Aruba', 'Bahrain', 'Barbados', 'Bermuda', 'Bonaire Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'British Virgin Islands', 'Cape Verde', 'Central African Republic', 'Christmas Island', 'Comoros', 'Cook Islands', "Cote d'Ivoire", 'Curacao', 'Democratic Republic of Congo', 'Dominica', 'Dominican Republic', 'Equatorial Guinea', 'Eswatini', 'Faeroe Islands', 'French Equatorial Africa', 'French Guiana', 'French Polynesia', 'French West Africa', 'Grenada', 'Guadeloupe', 'Hong Kong', 'Kiribati', 'Leeward Islands', 'Liechtenstein', 'Macao', 'Maldives', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritius', 'Mayotte', 'Micronesia (country)', 'Montserrat', 'Nauru', 'Niue', 'North Macedonia', 'Palau', 'Panama Canal Zone', 'Reunion', 'Ryukyu Islands', 'Saint Helena', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', 'Samoa', 'Sao Tome and Principe', 'Seychelles', 'Singa

In [14]:
''' manually go though the two lists and see which ones are the same country. 
The world dataset values will be replaced to allow for the co2 dataset updates '''

list1 = ['Bosnia and Herz.','Central African Rep.',"Côte d'Ivoire",'Dominican Republic','Equatorial Guinea','Solomon Islands', 'South Sudan', 'Timor', 'United States',]
list2=['Bosnia and Herzegovina','Central African Republic',"Cote d'Ivoire",'Dominican Rep.','Eq. Guinea', 'Solomon Is.', 'S. Sudan', 'Timor-Leste', 'United States of America',]

for i in range(len(list1)):
    world['countries'] = world['country'].replace(to_replace=list2[i], value=list1[i])

In [15]:
''' Merge the two datasets including only the countries that are in both of the datasets'''
world_co2 = world.merge(df_country, how="inner", on= ['country'])


We have now created a geopandas dataframe that can be used for plotting in Geopandas. Instead we will be using Holoviz Geoviews as this will give an interactive map.


In [16]:
world_co2

,pop_est,continent,country,iso_a3,gdp_md_est,geometry,countries,iso_code,year,co2,...,ghg_excluding_lucf_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji,FJI,1950,0.121,...,0.000,0.00,0.000,0.00,0.000,288994.0,0.000000e+00,0.000,0.000,0.000
1,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji,FJI,1951,0.132,...,0.000,0.00,0.000,0.00,0.000,296250.0,0.000000e+00,0.000,0.000,0.000
2,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji,FJI,1952,0.158,...,0.000,0.00,0.000,0.00,0.000,304883.0,0.000000e+00,0.000,0.000,0.000
3,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji,FJI,1953,0.169,...,0.000,0.00,0.000,0.00,0.000,314452.0,0.000000e+00,0.000,0.000,0.000
4,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji,FJI,1954,0.158,...,0.000,0.00,0.000,0.00,0.000,324583.0,0.000000e+00,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17664,1218208,North America,Trinidad and Tobago,TTO,43570.0,"POLYGON ((-61.68000 10.76000, -61.10500 10.890...",Trinidad and Tobago,TTO,2016,40.052,...,16.580,1.26,0.915,0.39,0.283,1377563.0,3.560577e+10,197.232,143174.722,5.539
17665,1218208,North America,Trinidad and Tobago,TTO,43570.0,"POLYGON ((-61.68000 10.76000, -61.10500 10.890...",Trinidad and Tobago,TTO,2017,39.948,...,16.668,1.27,0.918,0.39,0.282,1384060.0,3.478257e+10,209.285,151210.889,6.017
17666,1218208,North America,Trinidad and Tobago,TTO,43570.0,"POLYGON ((-61.68000 10.76000, -61.10500 10.890...",Trinidad and Tobago,TTO,2018,40.475,...,16.477,1.28,0.921,0.39,0.281,1389841.0,3.469735e+10,198.092,142528.854,5.709
17667,1218208,North America,Trinidad and Tobago,TTO,43570.0,"POLYGON ((-61.68000 10.76000, -61.10500 10.890...",Trinidad and Tobago,TTO,2019,40.378,...,0.000,0.00,0.000,0.00,0.000,1394969.0,0.000000e+00,197.438,141535.436,0.000


# Plotting using Geoviews

Now we will plot the data using geoviews. Let's choose the data for the year 2020, and when we hover over the country let us see both the $CO_2$ emissions and the country name. In vdims we need to put the column that we want the colour palette to work on first. We will alos be using the Robinson cartography projection, more can be found at https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html 

In [17]:
gv.Polygons(world_co2[world_co2['year']==2020], vdims=['co2','country']).opts(projection=ccrs.Robinson(), width=400, tools=['hover'])

:Polygons   [Longitude,Latitude]   (co2,country)

### Add Interactive Widgets to the Graph
Using Holoviz widgets helps interact with the data easily with a little coding. We will add a year slider and a dropdown box to select which column will project on the map. 

This can also be added to a dashboard via Holoviz Panel.

In [18]:
''' create definition for how the map interactives using widgets, nb. map color works on first vdim'''
def fn(slider, select): return gv.Polygons(world_co2[world_co2['year']==slider], vdims=[select,'country']).opts(projection=ccrs.Robinson(), width=400, tools=['hover'])

''' create a slider and a dropdown box''' 
slider = pn.widgets.IntSlider(value=1990, start=df.year.min(), end=df.year.max())
select = pn.widgets.Select(options=list(world_co2.columns), value=world_co2.columns[0])

''' Bind the widgets to the function '''
bound_fn = pn.bind(fn, slider, select)

In [19]:
''' show the widgets along side the plot '''
pn.Row(pn.Column(slider,select), bound_fn)

Row
    [0] Column
        [0] IntSlider(end=2020, start=1750, value=1990)
        [1] Select(options=['pop_est', 'continent', ...], value='pop_est')
    [1] ParamFunction(function)

## Creating a Continent Dataframe

We already have the continent values in the $CO_2$ dataset, all we need to do is get the geometry for each continent. Luckily this is easy with geopandas and its dissolve function

In [21]:
''' get continent data from co2 dataset '''
df_continent = df[df['country'].isin(continents)]

In [22]:
''' get only the continent and geometry columns and combine all rows by continent ''' 
world_continents = world[['continent','geometry']].dissolve(by='continent')

In [23]:
world_continents

,geometry
continent,
Africa,"MULTIPOLYGON (((-16.71373 13.59496, -17.12611 ..."
Antarctica,"MULTIPOLYGON (((-180.00000 -84.71338, -179.942..."
Asia,"MULTIPOLYGON (((27.19238 40.69057, 26.35801 40..."
Europe,"MULTIPOLYGON (((-177.66358 71.13277, -178.6937..."
North America,"MULTIPOLYGON (((-169.52944 62.97693, -170.2905..."
Oceania,"MULTIPOLYGON (((-179.79332 -16.02088, -179.917..."
Seven seas (open ocean),"POLYGON ((68.93500 -48.62500, 69.58000 -48.940..."
South America,"MULTIPOLYGON (((-71.37525 -17.77380, -71.46204..."


Next is to merge both datasets together. Note that the world_continents is merged on its index. We also use right inner merge to ensure that the world_continents data is added to each row of the df_continent dataset. This dataset also include the world data which will be removed as it has no geometry information.

In [24]:
''' create a new dataset which includes '''
continent_co2 = world_continents.merge(df_continent, how="right", left_on= ['continent'], right_on=['country'])

In [25]:
''' drop any rows that has no geometry data '''
continent_co2.dropna(axis=0, inplace = True)
type(continent_co2)

geopandas.geodataframe.GeoDataFrame

Note that we have to ensure that the dataframe is a geopandas type. If it is a pandas dataframe then the graphing will not work. In this case convert the dataframe using `df = geopandas.GeoDataFrame(df)`

### Plotting using Hvplot

Another good plotting tool is the hvplot from holoviz. This will use geopandas in the background to create the map. https://pyviz-dev.github.io/hvplot/reference/geopandas/polygons.html

In [29]:
continent_co2[continent_co2['year']==2020].hvplot.polygons(geo=True, hover_cols='all')

:Polygons   [Longitude,Latitude]   (index,iso_code,country,year,co2,co2_per_capita,trade_co2,cement_co2,cement_co2_per_capita,coal_co2,coal_co2_per_capita,flaring_co2,flaring_co2_per_capita,gas_co2,gas_co2_per_capita,oil_co2,oil_co2_per_capita,other_industry_co2,other_co2_per_capita,co2_growth_prct,co2_growth_abs,co2_per_gdp,co2_per_unit_energy,consumption_co2,consumption_co2_per_capita,consumption_co2_per_gdp,cumulative_co2,cumulative_cement_co2,cumulative_coal_co2,cumulative_flaring_co2,cumulative_gas_co2,cumulative_oil_co2,cumulative_other_co2,trade_co2_share,share_global_co2,share_global_cement_co2,share_global_coal_co2,share_global_flaring_co2,share_global_gas_co2,share_global_oil_co2,share_global_other_co2,share_global_cumulative_co2,share_global_cumulative_cement_co2,share_global_cumulative_coal_co2,share_global_cumulative_flaring_co2,share_global_cumulative_gas_co2,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,total_ghg,ghg_per_capita,total_ghg_excluding_lucf,ghg_excluding_lucf_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp)

In [34]:
''' create definition for how the map interactives using widgets, nb. map color works on first vdim'''
def continent_map(slider, select): return continent_co2[continent_co2['year']==slider].hvplot.polygons(geo=True, c=select, hover_cols='all')

''' Bind the widgets to the function '''
bound_continent_map = pn.bind(continent_map, slider, select)

''' show the widgets along side the plot '''
pn.Row(pn.Column(slider,select), bound_continent_map)

Row
    [0] Column
        [0] IntSlider(end=2020, start=1750, value=1990)
        [1] Select(options=['pop_est', 'continent', ...], value='coal_co2')
    [1] ParamFunction(function)